<a href="https://colab.research.google.com/github/klaudia-pruchnik/Proptify/blob/main/algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive/')

!ln -s /content/gdrive/My\ Drive/ ./mydrive

%cd /content/gdrive/MyDrive/projekt_inzynierski

/
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
ln: failed to create symbolic link './mydrive/My Drive': File exists
/content/gdrive/MyDrive/projekt_inzynierski


In [ ]:
!pip install sentence-transformers -q
!pip install -q faiss-cpu sentence-transformers
!python -m spacy download en_core_web_sm
!python -m spacy download pl_core_news_sm
!pip install gliner
!python -m spacy download en_core_web_md
!python -m spacy download pl_core_news_lg
!pip install spacy gliner transformers torch

In [ ]:
import faiss
import torch
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from math import log
from numpy.linalg import norm
import re
from gliner import GLiNER
import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans

Modele

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_e5 = SentenceTransformer('intfloat/multilingual-e5-base', device=device)
model_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

Dataframy

In [ ]:
df_songs = pd.read_parquet("./df_full_with_embeddings.parquet")

In [ ]:
df_tag_embeddings = pd.read_parquet("df_unique_tag_embeddings.parquet")
df_tag_embeddings

Embeddingi tagów

In [ ]:
TAG_VECS = np.array(df_tag_embeddings["tag_embedding"].to_list(), dtype=np.float32)
TAGS = df_tag_embeddings["tag"].tolist()

## Data preparation dodatkowe

zamień tag_list na listę

In [ ]:
def _ensure_list(x):
    if isinstance(x, list): return x
    if pd.isna(x) or x is None: return []
    return [t.strip() for t in str(x).split(",") if t.strip()]

df_songs["tags_list"] = df_songs["tags"].apply(_ensure_list)
df_songs["tag_count"] = df_songs["tags_list"].apply(len)
df_songs.head(5)

Uzupełnij tagi ich tagami nadrzędnymi dla lepszego dopasowania (to trzeba przenieść do innego pliku i poprawić bazę)

In [ ]:
# reguły: subgatunek -> nadrzędny
ALSO_ADD_PARENT = {
    # rock
    "progressive rock": "rock",
    "classic rock": "rock",
    "indie rock": "rock",
    "hard rock": "rock",
    "pop rock": "rock",
    "psychedelic rock": "rock",
    "punk rock": "rock",
    "blues rock": "rock",
    "post rock": "rock",

    # metal
    "heavy metal": "metal",
    "death metal": "metal",
    "black metal": "metal",
    "doom metal": "metal",
    "thrash metal": "metal",
    "melodic death metal": "metal",
    "symphonic metal": "metal",
    "gothic metal": "metal",
    "nu metal": "metal",
    "progressive metal": "metal",
    "power metal": "metal",
    "metalcore": "metal",

    # pop / electronic
    "indie pop": "pop",
    "synthpop": "pop",
    "drum and bass": "electronic",
}

# 2) reguły: słowa-klucze → nadrzędny
PARENT_KEYWORDS = {
    "rock": ["rock"],
    "metal": ["metal"],
    "pop": ["pop", "britpop"],
    "hip hop": ["hip hop", "rap"],
    "electronic": ["electronic", "techno", "house", "trance", "idm", "downtempo", "electro", "ambient"],
    "jazz": ["jazz"],
    "classical": ["classical"],
    "punk": ["punk"],
    "folk": ["folk"],
    "blues": ["blues"],
    "country": ["country"],
    "reggae": ["reggae"],
}

def expand_tags(tag_list):
    if tag_list is None or (isinstance(tag_list, float) and pd.isna(tag_list)):
        tag_list = []

    # normalizacja minimalna: małe litery, zamiana _ → spacja
    tags = {str(t).lower().replace("_", " ").strip() for t in tag_list}

    # 1) subgatunek → nadrzędny
    for child, parent in ALSO_ADD_PARENT.items():
        if child in tags:
            tags.add(parent)

    # 2) słowa kluczowe → nadrzędny
    for parent, kws in PARENT_KEYWORDS.items():
        if any(kw in t for t in tags for kw in kws):
            tags.add(parent)

    return sorted(tags)

# zastosowanie
df_songs["tags_list"] = df_songs["tags_list"].apply(expand_tags)


In [ ]:
df_songs["tags_list"] = df_songs["tags_list"].apply(
    lambda lst: [tag.replace("_", " ") for tag in lst]
)

# Algorytm dopasowania utworów

Budowanie inverted index do późniejszego scorowania utworów

In [ ]:
df_songs = df_songs.reset_index(drop=True)

def build_inverted_index(df_songs: pd.DataFrame, tags_col: str = "tags_list"):
    inv = defaultdict(list)
    df_count = defaultdict(int)  # document frequency: w ilu utworach wystąpił tag

    for i, tags in enumerate(df_songs[tags_col]):
        if not tags:
            continue
        seen = set()
        for t in tags:
            inv[t].append(i)
            if t not in seen:
                df_count[t] += 1
                seen.add(t)

    # zamieniamy listy na numpy dla szybkości
    for t in inv:
        inv[t] = np.asarray(inv[t], dtype=np.int32)

    return inv, df_count

INV_INDEX, DF_COUNT = build_inverted_index(df_songs, tags_col="tags_list")
N_SONGS = len(df_songs)
AVG_TAG_LEN = float(df_songs["tag_count"].mean()) if "tag_count" in df_songs else 10.0

Config do wszystkich parametrów algorytmu

In [ ]:
RETRIEVAL_CONFIG = {
    "n_candidates": 200,    # początkowe top najlepiej dopasowanych wg FAISS
}

SCORE_TIERS_CONFIG = {
    "t_high": 0.9,
    "t_mid": 0.7,
    "min_final": 100,         # chcemy finalnie tyle
    "max_c_from_low_tier": 15,  # ile max brać z Tier C gdy brakuje
}

POPULARITY_CONFIG = {
    "p_high": 70,
    "p_mid": 35,
    # procentowy miks w finalnym secie (docelowy)
    "mix": {
        "high": 0.4,     # popularne
        "mid": 0.35,     # średnie
        "low": 0.25,     # niszowe
    },
    "forced_popular": 2,    # ile utworów bardzo popularnych wstawić na sztywno
    "forced_popular_min": 80,
}

SAMPLING_CONFIG = {
    "final_n": 15,
    "alpha": 2.0,   # jak mocno faworyzujemy wyższy score przy losowaniu
}

QUERY_TAGS_CONFIG = {
    "rel_keep": 0.98,   # próg względny: bierzemy tagi >= 70% najlepszego
    "abs_keep": 0.9,  # próg absolutny: odetnij totalny szum
    "min_keep": 1,     # min liczba tagów w profilu
    "max_keep": 12,    # max liczba tagów w profilu
    "top_m_per_ngram": 2,
    "min_ngram_sim": 0.7,
    "min_unigram_sim": 0.5,
    "power": 1,
    "include_unigrams": True,
}

TAG_SCORING_CONFIG = {
    "use_idf": False,           # waż tagi rzadkie wyżej
    "k1": 1.2,                 # siła normalizacji „długości dokumentu” (liczby tagów utworu)
    "b": 0.75,
    "len_norm": False,
    "query_pow": 1.0,          # możesz podnieść np. 1.2 by ostrzej różnicować wagi tagów z promptu
    "normalize_by_tags": False # alternatywna, prostsza normalizacja: score / sqrt(n_tags_utworu)
}


Wyciągamy podzielone, oczyszczone wyrażenia z propta usera

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect, DetectorFactory

In [ ]:
# Ustawiamy ziarno dla detekcji języka (żeby wyniki były powtarzalne dla krótkich tekstów)
DetectorFactory.seed = 0

In [ ]:
# spaCy: Model do parsowania gramatycznego i Matchera (zapewnia precyzyjne wzorce)
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

In [ ]:
GLINER_LABELS = [
    "opis_emocji",
    "poziom_energii",
    "opis_tempa",
    "gatunek_muzyczny",
    "opis_wokalu",
    "przeznaczenie_utworu",
    "cecha_brzmienia",
    "cecha_instrumentu"
]

GENERIC_LEMMAS = [
    # Polski
    "muzyka", "utwór", "piosenka", "kawałek", "playlista", "lista", "numer", "rok", "klimat", "styl",
    # Angielski
    "music", "song", "track", "playlist", "list", "number", "vibe", "tune", "genre", "style"
]

GENERIC_VERBS = [
    # --- POLSKI (Bezokoliczniki / Lematy) ---
    # Szukanie / Chcenie
    "szukać", "poszukiwać", "chcieć", "pragnąć", "potrzebować", "woleć", "wymagać",
    # Bycie / Posiadanie
    "być", "mieć", "znajdować", "znaleźć",
    # Słuchanie / Odtwarzanie
    "słuchać", "posłuchać", "usłyszeć", "grać", "zagrać", "puszczać", "puścić", "odtworzyć", "zapodać",
    # Prośby / Rekomendacje
    "prosić", "polecić", "polecać", "rekomendować", "sugerować", "zaproponować", "dawać", "dać",

    # --- ANGIELSKI (Base forms) ---
    # Searching / Wanting
    "search", "look", "find", "want", "need", "desire", "wish", "require",
    # Being / Having
    "be", "have", "get",
    # Listening / Playing
    "listen", "hear", "play", "replay", "stream",
    # Requests
    "give", "recommend", "suggest", "show", "provide",
]

NEGATION_TERMS = [
    # PL
    "nie", "bez", "mało", "zero", "ani", "żaden", "brak", "mniej",
    # EN
    "no", "not", "without", "less", "non", "neither", "nor", "lack", "zero"
]

In [ ]:
def create_matcher_for_nlp(nlp_instance):
    """Tworzy obiekt Matcher przypisany do konkretnego modelu językowego"""
    matcher = Matcher(nlp_instance.vocab)

    # Warunek wykluczający
    # To musi być Rzeczownik, ale NIE MOŻE być na liście generycznej
    noun_filter = {
        "POS": {"IN": ["NOUN", "PROPN"]},
        "IS_STOP": False,
        "LEMMA": {"NOT_IN": GENERIC_LEMMAS}
    }

    matcher.add("FRAZA", [
        # 1. Samodzielne NOUN/PROPN (rock)
        [noun_filter],
        # 2. ADJ + NOUN (szybki bas)
        [{"POS": "ADJ"}, noun_filter],
        # 3. (ADV)? + ADP + NOUN (z tańca, z klimatem -> filtrowane)
        [{"POS": "ADV", "OP": "?"}, {"POS": "ADP"}, noun_filter],
        # 4. ADV + ADJ (bardzo wesoła)
        [{"POS": "ADV"}, {"POS": "ADJ", "IS_STOP": False}],
        # 5. ADJ Samotny (rockowa)
        [{"POS": "ADJ", "IS_STOP": False}],
        # 6. Złożone Rzeczowniki (post rock)
        [{"POS": {"IN": ["NOUN", "PROPN"]}, "IS_STOP": False}, noun_filter],
        # 7. Złożona relacja (dobra do tańca)
        [{"POS": "ADV", "OP": "?"}, {"POS": "ADJ"}, {"POS": "ADP"}, noun_filter],
        # 8. Celuje w: Czasownik opisowy + Rzeczownik/Adjektive/Zaimek
        [
            {"POS": "VERB", "LEMMA": {"NOT_IN": GENERIC_VERBS}},
            {"POS": {"IN": ["NOUN", "ADJ", "PRON"]}, "OP": "+"} # Obiekt czasownika (może być więcej niż jedno słowo)
        ]
    ])
    return matcher

# Tworzymy matchery raz na starcie
matcher_pl = create_matcher_for_nlp(nlp_pl)
matcher_en = create_matcher_for_nlp(nlp_en)


In [ ]:
# 4. FUNKCJA POMOCNICZA: SPRAWDZANIE NEGACJI
# ----------------------------------------------------------------------

def is_span_negated(doc, start_index, window=2):
    """
    Sprawdza, czy przed frazą (start_index) stoi słowo przeczące.
    Patrzy 'window' tokenów wstecz.
    """
    lookback = max(0, start_index - window)
    preceding_tokens = doc[lookback:start_index]

    for token in preceding_tokens:
        if token.text.lower() in NEGATION_TERMS:
            return True
    return False

In [ ]:
# ----------------------------------------------------------------------
# 3. GŁÓWNA FUNKCJA HYBRYDOWA: EKSTRAKCJA FRAZ
# ----------------------------------------------------------------------

def extract_relevant_phrases(prompt):
    """
    Łączy GLiNER i Matcher w celu wydobycia istotnych, niepowtarzających się fraz
    opisujących cechy muzyczne z promptu.
    """
    prompt = prompt.lower()

    try:
        lang_code = detect(prompt)
    except:
        lang_code = 'pl'

    # Wybór odpowiedniego zestawu narzędzi
    if lang_code == 'en':
        current_nlp = nlp_en
        current_matcher = matcher_en
        lang_msg = "EN"
    else:
        current_nlp = nlp_pl
        current_matcher = matcher_pl
        lang_msg = "PL"

    # Przetwarzanie wybranym modelem
    doc = current_nlp(prompt)

    # 1. EKSTRAKCJA Z GLINER (Łapie kontekst, długie frazy, np. "muzyka, która jest smutna")
    # Niska wartość threshold jest celowa, aby złapać więcej kandydatów.
    gliner_entities = model_gliner.predict_entities(prompt, GLINER_LABELS, threshold=0.1)
    gliner_phrases_raw = [e['text'].lower() for e in gliner_entities]

    # 2. EKSTRAKCJA Z MATCHER (Łapie pojedyncze rzeczowniki i precyzyjne wzorce)
    matcher_matches = current_matcher(doc)
    matcher_spans = [doc[start:end] for match_id, start, end in matcher_matches]

    # filter_spans wybiera najdłuższe, nie nakładające się frazy Matchera
    combined_spans = filter_spans(matcher_spans)
    matcher_phrases = [span.text.lower() for span in combined_spans]

    # 3. FILTRACJA WYNIKÓW GLINER
    filtered_gliner_phrases = []

    for phrase in gliner_phrases_raw:
        phrase_doc = current_nlp(phrase)
        is_generic_noise = False

        # Sprawdzenie lematu GŁÓWNEGO słowa w frazie GLINERa
        for token in phrase_doc:
            if token.pos_ in ["NOUN", "PROPN"] and token.lemma_.lower() in GENERIC_LEMMAS:
                # Jeśli fraza zawiera generyczny rzeczownik, traktujemy ją jako szum
                is_generic_noise = True
                break

        if not is_generic_noise:
            filtered_gliner_phrases.append(phrase)

    # 4. Fuzja wyników
    all_phrases = filtered_gliner_phrases + matcher_phrases
    unique_phrases = sorted(list(set([p.strip() for p in all_phrases if len(p.strip()) > 1])))

    print(f'Matcher only: {matcher_phrases}')
    print(f'Gliner only: {filtered_gliner_phrases}')
    print(f"[{lang_msg}] Prompt: '{prompt}' \n-> {unique_phrases}")

    return unique_phrases

In [ ]:
# ----------------------------------------------------------------------
# 4. PRZYKŁADY UŻYCIA
# ----------------------------------------------------------------------

TEST_PROMPT_1 = "szukam muzyki rockowej, ale takiej pełnej spokoju i bardzo wesołej, trochę do tańca"
TEST_PROMPT_2 = "rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii"
TEST_PROMPT_3 = "rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii, post rock, alternative rock, rock alternatywny, pop"
TEST_PROMPT_4 = "muzyka rockowa z lat 90, z klimatem podróży, postpankowa"
TEST_PROMPT_5 = "muzyka bez słów, smutna, nostalgiczna, spokojna"
TEST_PROMPT_6 = "szybka, intensywna, bardzo dobra do tańca, zabawy, zajsta do tańca, idealna do tańca, "
TEST_PROMPT_7 = "albo zwykły rock, albo jakiś post rock albo punk, coś takiego"
TEST_PROMPT_8 = "muzyka dynamiczna, szybko, szybkie tempo, wysokie tempo, energiczna"
TEST_PROMPT_9 = "muzyka, która koi nerwy"
TEST_PROMPT_10 = "muzyka, która koi nerwy"
TEST_PROMPT_11 = "Szukam muzyki rockowej, ale nie smutnej"
TEST_PROMPT_12 = "I want energetic songs, no slow music"

relevant_phrases = extract_relevant_phrases(TEST_PROMPT_12)